In [1]:
import numpy as np
import pandas as pd
from arch import arch_model
from models import ArchModel
import statsmodels.api as sm

In [2]:
df = pd.read_excel('../CAViaR project/data/GM Equity.xlsx')
df.columns = df.iloc[5].values
df = df.iloc[6:].iloc[::-1].reset_index(drop=True)
df['Log Return'] = df.PX_LAST.apply(lambda x: np.log(x)).diff()
df.head()

,Date,PX_LAST,Change,% Change,PX_OPEN,Change,% Change,PX_HIGH,Change,% Change,PX_LOW,Change,% Change,Log Return
0,2010-11-17 00:00:00,25.3909,NaN,NaN,25.3909,NaN,NaN,25.3909,NaN,NaN,25.3909,NaN,NaN,NaN
1,2010-11-18 00:00:00,26.3066,NaN,NaN,26.9298,NaN,NaN,27.6915,NaN,NaN,26.0757,NaN,NaN,0.035429
2,2010-11-19 00:00:00,26.3604,NaN,NaN,26.2758,NaN,NaN,26.5451,NaN,NaN,25.4756,NaN,NaN,0.002043
3,2010-11-22 00:00:00,26.2219,NaN,NaN,26.3142,NaN,NaN,26.5297,NaN,NaN,26.0142,NaN,NaN,-0.005268
4,2010-11-23 00:00:00,25.5833,NaN,NaN,26.1219,NaN,NaN,26.1527,NaN,NaN,25.5371,NaN,NaN,-0.024655


In [3]:
log_ret_100x = 100 * (df['Log Return'].dropna() - df['Log Return'].dropna().mean()).reset_index(drop=True)

In [4]:
arch = ArchModel(9)

In [5]:
cus_res = arch.fit(log_ret_100x)

In [6]:
cus_res

      fun: -25544.56962265682
 hess_inv: <10x10 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.       , -0.0003638, -0.0003638, -0.0003638, -0.0003638,
       -0.0003638, -0.0003638, -0.0003638, -0.0003638, -0.0003638])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 352
      nit: 30
     njev: 32
   status: 0
  success: True
        x: array([1.77224126e+07, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00])

In [7]:
cus_res

      fun: 6423.789372214314
 hess_inv: <10x10 LbfgsInvHessProduct with dtype=float64>
      jac: array([-7.36690713e-03,  1.00953912e-02,  1.68729457e+00,  7.54880602e-03,
        3.11956683e-02,  1.93722372e-02,  1.15505827e-02, -6.09361450e-03,
        1.65528036e-02,  1.63709046e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 242
      nit: 17
     njev: 22
   status: 0
  success: True
        x: array([1.80299281, 0.06383229, 0.        , 0.03106833, 0.07271849,
       0.06842066, 0.06925327, 0.03982586, 0.11416383, 0.14832313])

In [8]:
am = arch_model(log_ret_100x, vol='ARCH', p=9)

In [9]:
res = am.fit()

Iteration:      1,   Func. Count:     13,   Neg. LLF: 12914.669666422396
Iteration:      2,   Func. Count:     28,   Neg. LLF: 474994.93057286204
Iteration:      3,   Func. Count:     41,   Neg. LLF: 711847.5978756085
Iteration:      4,   Func. Count:     55,   Neg. LLF: 6948.0606166662
Iteration:      5,   Func. Count:     69,   Neg. LLF: 6866.975308577372
Iteration:      6,   Func. Count:     83,   Neg. LLF: 6845.506703461275
Iteration:      7,   Func. Count:     97,   Neg. LLF: 6885.91996803865
Iteration:      8,   Func. Count:    111,   Neg. LLF: 6614.752816457122
Iteration:      9,   Func. Count:    124,   Neg. LLF: 6696.86940482588
Iteration:     10,   Func. Count:    138,   Neg. LLF: 6556.214295523846
Iteration:     11,   Func. Count:    151,   Neg. LLF: 6451.893242881393
Iteration:     12,   Func. Count:    164,   Neg. LLF: 6516.460923757972
Iteration:     13,   Func. Count:    178,   Neg. LLF: 6450.7484071110875
Iteration:     14,   Func. Count:    191,   Neg. LLF: 6510.687417

In [ ]:
print(res.summary())

In [ ]:
y_df = pd.concat([(log_ret_100x/100) ** 2, ((log_ret_100x/100) ** 2).shift(1)], axis=1)
y_df.columns = ['t', 't-1']
y_df = y_df.dropna()
y_df

In [ ]:
mod = sm.OLS(y_df['t'], sm.add_constant(y_df['t-1']))

In [ ]:
ols_res = mod.fit()

In [ ]:
print(ols_res.summary())